In [1]:
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-11-17 20:44:37.415602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-17 20:44:37.475278: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Num GPUs Available:  1


2024-11-17 20:44:38.454006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-17 20:44:38.475833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-17 20:44:38.476004: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import tensorflow as tf

USE_GPU = True

if USE_GPU and tf.config.list_physical_devices('GPU'):
    device = 'GPU'
    print("Using GPU")
else:
    device = 'CPU'
    print("Using CPU")

Using GPU


In [3]:
import tensorflow_datasets as tfds

# Load the ImageNet dataset

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'

# Load the ImageNet Resized dataset (e.g., 32x32)
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

# Split the dataset into training and validation sets
train_dataset = dataset['train']
val_dataset = dataset['validation']

# Print the dataset info
print(info)

/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tfds.core.DatasetInfo(
    name='imagenet_resized',
    full_name='imagenet_resized/64x64/0.1.0',
    description="""
    This dataset consists of the ImageNet dataset resized to fixed size. The images
    here are the ones provided by Chrabaszcz et. al. using the box resize method.
    
    For [downsampled ImageNet](http://image-net.org/download.php) for unsupervised
    learning see `downsampled_imagenet`.
    
    those from the other ImageNet datasets provided by Tensorflow datasets. See the
    original
    [label list](https://github.com/PatrykChrabaszcz/Imagenet32_Scripts/blob/master/map_clsloc.txt),
    and the
    [labels used by this dataset](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/image_classification/imagenet_resized_labels.txt).
    Additionally, the original authors 1 index there labels which we convert to 0
    indexed by subtracting one.
    """,
    config_description="""
    Images resized to 64x64
    """,
    homepage='https://patrykc

2024-11-15 18:48:19.540097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 18:48:19.541578: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-15 18:48:19.541893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-15 18:48:19.542032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), bu

In [2]:
# from keras.applications import VGG16
from VGG16.vgg import VGG
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

alpha = 0.03
temperature = 0.02

# Load the VGG16 model
model = VGG(alpha=alpha, temperature=temperature)


/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import preprocess_input

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

train_dataset = dataset['train']
val_dataset = dataset['validation']


def preprocess(image, label):
    image = tf.image.resize(image, [64, 64])
    image = preprocess_input(image) 
    return image, label

batch_size = 32
train_dataset = (train_dataset
                 .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                 .shuffle(1000)  
                 .batch(batch_size)
                 .prefetch(tf.data.AUTOTUNE))

val_dataset = (val_dataset
               .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(batch_size)
               .prefetch(tf.data.AUTOTUNE))

In [16]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

data_dir = '/home/asko/Downloads'

transform = transforms.Compose([
    transforms.Resize((227,227)),  # Resize images to a size suitable for VGG16
    # transforms.CenterCrop(224),  # Crop to the expected input size for VGG16
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per VGG16
])

train_dataset = datasets.ImageNet(root=data_dir, split='train', transform=transform)

subset_indices = list(range(16000))  
train_subset = Subset(train_dataset, subset_indices)

batch_size = 32
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)

for images, labels in train_loader:
    print(labels.shape)


torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])


KeyboardInterrupt: 

In [4]:
train_loader

In [5]:
import tensorflow as tf
device = 'cuda' if len(tf.config.list_physical_devices('GPU'))!=0 else 'cpu'
model = model.to(device)
print(device)

cuda


In [6]:
def check_accuracy_part34(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    print(device)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            print(y)
            print(preds)
            # print(scores)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        # print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc


In [7]:
def check_accuracy_part35(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # Set model to evaluation mode
    print(device)
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # Move inputs to device
            y = y.to(device=device, dtype=torch.long)  # Move labels to device, long for classification
            
            scores = model(x)  # Get model predictions
            _, preds = scores.max(1)  # Get predicted class indices
            
            num_correct += preds.eq(y).sum().item()  # Compare predictions with ground truth
            num_samples += preds.size(0)  # Count samples

        acc = float(num_correct) / num_samples
        return 100 * acc


In [ ]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                # check_accuracy_part34(loader_val, model)
                print()

In [8]:
def one_hot_encode(labels, num_classes=1000):
    # Initialize a tensor of zeros with the shape [batch_size, num_classes]
    one_hot_labels = torch.zeros(labels.size(0), num_classes).cuda()
    one_hot_labels.scatter_(1, labels.view(-1, 1), 1)  # Set 1 at the correct index
    return one_hot_labels

In [14]:
import torch.optim as optim
import torch
import torch.nn.functional as F

# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-5, momentum=0.9, nesterov=True)
torch.set_grad_enabled(True)

num_epochs = 10
batch_size = 32  
model_weights = None

for epoch in range(num_epochs):
    total_loss = 0.0 
    if model_weights is None:
        model_weights = model.conv1_1.weight.clone()   
    for batch_idx, (images, labels) in enumerate(train_loader):
        model.train()           
        
        images = images.to(device, dtype=torch.float32)
        labels = labels.to(device, dtype=torch.long)
        
        output = model(images)
        
        loss = F.cross_entropy(input=output, target=labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print(torch.sum(model_weights - model.conv1_1.weight))   
    model_weights = model.conv1_1.weight.clone()        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / (batch_idx+1)}, Training Accuracy: {check_accuracy_part34(train_loader, model)}")

tensor(-0.0165, device='cuda:0', grad_fn=<SumBackward0>)
cuda
tensor([ 0, 11,  9,  1,  4,  4,  7,  5,  8,  9,  4,  1,  9, 12,  2,  8,  9,  6,
         0,  4, 12,  6,  3,  0,  7,  7,  0, 10,  8,  9,  4,  5],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
tensor([ 0,  1, 11,  4,  2,  4,  7,  5,  4, 11,  8,  4, 10, 10, 12,  5, 10,  5,
         7,  1,  7,  6,  8,  3,  1,  8,  2,  9,  1,  4, 10,  6],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
tensor([ 3,  8,  9,  6, 10,  7,  2, 10,  0, 11,  8,  2, 10, 11, 10,  1, 11,  9,
         3,  6, 12,  1,  1, 11, 10,  9,  3,  0, 12, 10,  1,  1],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 1

KeyboardInterrupt: 

In [20]:
check_accuracy_part34(train_loader, model)

cuda
Got 1300 / 16000 correct (8.12)
